In [1]:
import numpy as np
import pandas as pd
import time
import holdem_calc
import parallel_holdem_calc
from datetime import datetime


In [2]:
def import_and_standardize_time_FGs(subject_num_):
    FGs = pd.read_csv(subject_num_ + "_FGs_withcorrecttimestamps.csv")
    FGs = FGs.drop('Unnamed: 0', 1)
    start_time = datetime.strptime(PokerActs_from_game.iloc[0,2], "%H:%M:%S,%f")
    FGs['RelativeTime'] = 0
    FGs['RelativeSec'] = 0
    
    for i in range (0, len(FGs)):
    
        if len(FGs.iloc[i,-3])>19:
            time = datetime.strptime(FGs.iloc[i,-3][11:], "%H:%M:%S.%f")
        else:
            time = datetime.strptime(FGs.iloc[i,-3][11:], "%H:%M:%S")

        difference = time - start_time
        FGs.iloc[i,-2] = difference
        FGs.iloc[i,-1] = (difference.seconds) + (difference.microseconds/1000000) #+ (difference.minutes*60)
    
    return FGs

In [3]:
def get_preflop_win_analysis(poker_actions_):
    preflop_stats = pd.DataFrame.from_csv('PreflopOdds.tsv', sep='\t', header=0)
    poker_actions_['Win%'] = "?"
    
    for i in range(0, len(poker_actions_)):
        if (poker_actions_.iloc[i,6] == "PREFLOP" and poker_actions_.iloc[i,10] != "[?s,?h]"):
            preflop_string = convert_preflop_hand(poker_actions_.iloc[i,10])
            win_odds = preflop_stats.loc[preflop_stats['Name'] == preflop_string, 'Win %'].iloc[0]
            poker_actions_.iat[i,13] = (win_odds/100)

    return poker_actions_

In [4]:
def convert_preflop_hand(preflop_hand):
    first_suit = ""
    second_suit = ""
    first_val = ""
    second_val = ""
    first_key = ""
    second_key = ""
    s_or_o = ""
    
    faces = {"T" : 10, "J": 11, "Q": 12, "K": 13, "A": 14}

    for i, c in enumerate(preflop_hand):
        if (i==1):
            first_val = c
        if (i==2):
            first_suit = c
        if (i==4):
            second_val = c
        if (i==5):
            second_suit = c
    just_cards = first_val+first_suit+second_val+second_suit

    if (first_suit == second_suit):
        s_or_o = "s"
    else:
        s_or_o = "o"
    if (first_val in faces):
        first_key = faces[first_val]
    else:
        first_key = int(first_val)
    if (second_val in faces):
        second_key = faces[second_val]
    else:
        second_key = int(second_val)
        
    vals = [first_key,second_key]
    vals.sort(key=int, reverse=True)
    if(vals[0] > 9):
        alpha_key = (list(faces.keys())[list(faces.values()).index(vals[0])])
    else:
        alpha_key = vals[0]
    if(vals[1] > 9):
        beta_key = (list(faces.keys())[list(faces.values()).index(vals[1])])
    else:
        beta_key = vals[1]
    if (alpha_key in faces):
        first_val = alpha_key
    else:
        first_val = vals[0]
    if (beta_key in faces):
        second_val = beta_key
    else:
        second_val = vals[1]
    preflop = str(first_val)+str(second_val)+s_or_o

    return(preflop)




In [5]:
def get_postflop_win_analysis(poker_actions_):
    previous_phase = "PREFLOP"
    hand_win_odds = ""
    already_calculated = False
    for i in range(0, len(poker_actions_)):
        current_phase = poker_actions_.iloc[i,6]
        if (poker_actions_.iloc[i,6] != "PREFLOP" and already_calculated == True):
            poker_actions_.iloc[i,13] = hand_win_odds

        if (current_phase != previous_phase):
            already_calculated = False
            if (poker_actions_.iloc[i,6] != "PREFLOP" and poker_actions_.iloc[i,10] != "[?s,?h]"):
                print("GETTING NEW WIN % at phase " + repr(current_phase))
                raw_board_cards = poker_actions_.iloc[i,11]
                if(len(raw_board_cards)==10):
                    board_cards = [str(raw_board_cards[1:3]),str(raw_board_cards[4:6]),str(raw_board_cards[7:9])]
                if(len(raw_board_cards)==13):
                    board_cards = [str(raw_board_cards[1:3]),str(raw_board_cards[4:6]),str(raw_board_cards[7:9]), str(raw_board_cards[10:12])]
                if(len(raw_board_cards)==16):
                    board_cards = [str(raw_board_cards[1:3]),str(raw_board_cards[4:6]),str(raw_board_cards[7:9]), str(raw_board_cards[10:12]),str(raw_board_cards[13:15])]
                holes = poker_actions_.iloc[i,10]
                hole_one = str(holes[1:3])
                hole_two = str(holes[4:6])
                win_odds = holdem_calc.calculate(board_cards, True, 1, None, [hole_one, hole_two, "?", "?"], False)
                print(win_odds[1])
                hand_win_odds = win_odds[1]
                poker_actions_.iloc[i,13] = hand_win_odds
                already_calculated = True

        previous_phase = current_phase
            
    return(poker_actions_)
        

In [6]:
def get_win_odds(poker_acts_):
    poker_wins = get_preflop_win_analysis(poker_acts_)
    poker_wins = get_postflop_win_analysis(poker_wins)
    
    return poker_wins

In [7]:
def import_and_standardize_time_PokerActs(subject_num_):
    
    PokerActs = pd.read_csv(subject_num_ + "_pokeracts_withcorrecttimestamps.csv")
    PokerActs['Hole Cards'] = PokerActs['Hole Cards'].fillna('[?s,?h]')
    PokerActs['Board Cards'] = PokerActs['Board Cards'].fillna('')
    PokerActs = get_win_odds(PokerActs)
    start_time = datetime.strptime(PokerActs.iloc[0,2], "%H:%M:%S,%f")

    PokerActs['RelativeTime'] = 0
    PokerActs['RelativeSec'] = 0
    
    for i in range (0, len(PokerActs)):
        time = datetime.strptime(PokerActs.iloc[i,2], "%H:%M:%S,%f")

        difference = time - start_time
        PokerActs.iloc[i,-2] = difference
        PokerActs.iloc[i,-1] = (difference.seconds) + (difference.microseconds/1000000)
        
    return PokerActs

In [8]:
def calc_poker_stats(PokerActs_from_game_, hero_name_, villain_name_):
    PokerActs_from_game_['M Ratio'] = '?'
    PokerActs_from_game_['EV'] = '?'
    PokerActs_from_game_['Pot Odds'] = '?'
    PokerActs_from_game_['Pot Odds CV'] = '?'
    PokerActs_from_game_['Implied Odds'] = '?'
    PokerActs_from_game_['Fold %'] = '?'
    PokerActs_from_game_['Fold Equity'] = '?'
    PokerActs_from_game_['Semi-Bluff Fold %'] = '?'
    PokerActs_from_game_['Time to Act'] = '?'
    PokerActs_from_game_['Relative Bet'] = '?'
    PokerActs_from_game_['Call Stakes'] = '?'

    for i in range(0, len(PokerActs_from_game_)):
        if i > 0:
            PokerActs_from_game_.loc[i,'Time to Act'] = PokerActs_from_game_.loc[i,'RelativeSec'] - PokerActs_from_game_.loc[i-1,'RelativeSec']
            current_line = PokerActs_from_game_.loc[i,'Log Line']
            current_words = current_line.split()
            prev_line = PokerActs_from_game_.loc[i-1,'Log Line']
            words = prev_line.split()
            PokerActs_from_game_.loc[i,'M Ratio'] = (min(PokerActs_from_game_.loc[i,'Hero Chips'], PokerActs_from_game_.loc[i,'Villain Chips']))/(PokerActs_from_game_.loc[i,'Small Blind']+PokerActs_from_game_.loc[i,'Big Blind'])
            if hero_name_ in current_words and ("calls" in current_words or "bets" in current_words):
                call_amt = int(current_words[2][1:].strip('.'))
                PokerActs_from_game_.loc[i,'Pot Odds'] = call_amt/(PokerActs_from_game_.loc[i,'Pot Chips'])
                PokerActs_from_game_.loc[i,'Relative Bet'] = (call_amt/PokerActs_from_game_.loc[i,'M Ratio'])#(PokerActs_from_game_.loc[i,'Hero Chips'] + call_amt))
            if hero_name_ in current_words and ("bets" in current_words or ("all in with" in current_words and ((villain_name_ + " is all in with") not in words))):
                bet_amt = int(current_words[2][1:].strip('.'))
                PokerActs_from_game_.loc[i,'Fold %'] = bet_amt/(PokerActs_from_game_.loc[i,'Pot Chips'])
         
    
    #Calculating EV, Call Stakes, Pot Odds, and Implied Odds, and Semi-Bluff FOld%
            if PokerActs_from_game_.loc[i,'Win%'] != '?':
            
                if villain_name_ in words and "bets" in words:
                    facing_bet = (int(words[2][1:].strip('.')))
                    PokerActs_from_game_.loc[i,'EV'] = (PokerActs_from_game_.loc[i,'Win%']*PokerActs_from_game_.loc[i,'Pot Chips']) - (1-PokerActs_from_game_.loc[i,'Win%'])*facing_bet
                if villain_name_ in current_words and "bets" in current_words:
                    facing_bet = (int(current_words[2][1:].strip('.')))
                    PokerActs_from_game_.loc[i,'Call Stakes'] = facing_bet/PokerActs_from_game_.loc[i,'M Ratio']
                elif "PREFLOP" in current_words:
                    facing_bet = PokerActs_from_game_.loc[i,'Small Blind']
                    PokerActs_from_game_.loc[i,'EV'] = (PokerActs_from_game_.loc[i,'Win%']*PokerActs_from_game_.loc[i,'Pot Chips']) - (1-PokerActs_from_game_.loc[i,'Win%'])*facing_bet
                elif "FLOP" in current_words or "TURN" in current_words or "RIVER" in current_words:
                    facing_bet = 0
                    PokerActs_from_game_.loc[i,'EV'] = (PokerActs_from_game_.loc[i,'Win%']*PokerActs_from_game_.loc[i,'Pot Chips']) - (1-PokerActs_from_game_.loc[i,'Win%'])*facing_bet
                if hero_name_ in current_words and ("calls" in current_words or "bets" in current_words):

                    call_amt = int(current_words[2][1:].strip('.'))
                    PokerActs_from_game_.loc[i,'Pot Odds CV'] = float(PokerActs_from_game_.loc[i,'Win%']) - float(PokerActs_from_game_.loc[i,'Pot Odds'])
                    if PokerActs_from_game_.loc[i,'Win%'] != 0:
                        PokerActs_from_game_.loc[i,'Implied Odds'] = (call_amt/float(PokerActs_from_game_.loc[i,'Win%'])) - PokerActs_from_game_.loc[i,'Pot Chips']
            
                if hero_name_ in current_words and ("bets" in current_words or ("all in with" in current_words and ((villain_name_ + " is all in with") not in words))):
                    PokerActs_from_game_.loc[i,'Semi-Bluff Fold %'] = float(PokerActs_from_game_.loc[i,'Fold %']) - (1.5*(PokerActs_from_game_.loc[i,'Win%']))

    return PokerActs_from_game_

In [9]:
def name_poker_events(PokerActs_from_game_, hero_name_, villain_name_):
    PokerActs_from_game_['Win/Lose Events'] = 'Not Applicable'#'Toss Up'
    PokerActs_from_game_['Win/Loss'] = 'Not Applicable'
    PokerActs_from_game_['New Card Events'] = 'Not Applicable'#'No News'
    PokerActs_from_game_['Calling Events'] = 'Not Applicable'#'Toss Up Call'
    PokerActs_from_game_['Betting Events'] = 'Not Applicable'#'Toss Up Bet'
    PokerActs_from_game_['Bet/Bluff'] = 'Not Applicable'
    PokerActs_from_game_['C/R/F'] = 'Not Applicable'
    
    for i in range(0, len(PokerActs_from_game_)):
        if PokerActs_from_game_.loc[i,'Win%'] != '?':
            current_line = PokerActs_from_game_.loc[i,'Log Line']
            current_words = current_line.split()
#             print("NOW POKER EVENTS")
#             print(current_words)

#                 previous_line = PokerActs_from_game_.loc[i-1,'Log Line']
#                 previous_words = previous_line.split()
#                 if float(PokerActs_from_game_.loc[i,'Win%']) > 0.75:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Favored Win'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) > 0.85:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Very Favored Win'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) < 0.35:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Lucky Win'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) < 0.15:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Very Lucky Win'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) < 0.35 and villain_name_ in previous_words and "folds" in previous_words:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Villain Bluffed Out'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) > 0.75 and villain_name_ in previous_words and "folds" in previous_words:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Villain Forced Out'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) >= 0.35 and float(PokerActs_from_game_.loc[i,'Win%'] <= 0.75):
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Toss Up Win'
            #Losses

#                 previous_line = PokerActs_from_game_.loc[i-1,'Log Line']
#                 previous_words = previous_line.split()
#                 if float(PokerActs_from_game_.loc[i,'Win%']) > 0.75:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Unlucky Loss'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) > 0.85:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Very Unlucky Loss'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) < 0.35:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Predictable Loss'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) < 0.15:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Very Predictable Loss'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) < 0.35 and hero_name_ in previous_words and "folds" in previous_words:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Hero Discretely Bows Out'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) > 0.75 and hero_name_ in previous_words and "folds" in previous_words:
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Hero Cedes Strong Position'
#                 if float(PokerActs_from_game_.loc[i,'Win%']) >= 0.35 and float(PokerActs_from_game_.loc[i,'Win%'] <= 0.75):
#                     PokerActs_from_game_.at[i,'Win/Lose Events'] = 'Toss Up Loss'
                
            #Preflop Reveals
            if "PREFLOP" in current_words:
                if float(PokerActs_from_game_.loc[i,'Win%']) > .60:
                    PokerActs_from_game_.at[i,'New Card Events'] = 'Good Deal'
                if float(PokerActs_from_game_.loc[i,'Win%']) < .35:
                    PokerActs_from_game_.at[i,'New Card Events'] = 'Lousy Deal'
                if float(PokerActs_from_game_.loc[i,'Win%']) >= 0.35 and float(PokerActs_from_game_.loc[i,'Win%'] <= 0.60):
                    PokerActs_from_game_.at[i,'New Card Events'] = 'Toss Up Deal'
        
            #Flop/Turn/River Reveals
            if "FLOP" in current_words or "TURN" in current_words or "RIVER" in current_words:
#                 if (float(PokerActs_from_game_.loc[i,'Win%']) - float(PokerActs_from_game_.loc[i-1,'Win%'])) > .25:
#                     PokerActs_from_game_.at[i,'New Card Events'] = 'Great Break'
                if (float(PokerActs_from_game_.loc[i,'Win%']) - float(PokerActs_from_game_.loc[i-1,'Win%'])) > .15:
                    PokerActs_from_game_.at[i,'New Card Events'] = 'Lucky Break'
                elif (float(PokerActs_from_game_.loc[i,'Win%']) - float(PokerActs_from_game_.loc[i-1,'Win%'])) > 0:
                    PokerActs_from_game_.at[i,'New Card Events'] = 'Positive Break'
#                 elif (float(PokerActs_from_game_.loc[i,'Win%']) - float(PokerActs_from_game_.loc[i-1,'Win%'])) < -.25:
#                     PokerActs_from_game_.at[i,'New Card Events'] = 'Awful Break'
                elif (float(PokerActs_from_game_.loc[i,'Win%']) - float(PokerActs_from_game_.loc[i-1,'Win%'])) < -.15:
                    PokerActs_from_game_.at[i,'New Card Events'] = 'Unlucky Break'
                elif (float(PokerActs_from_game_.loc[i,'Win%']) - float(PokerActs_from_game_.loc[i-1,'Win%'])) <= 0:
                    PokerActs_from_game_.at[i,'New Card Events'] = 'Negative Break'
                else:
                    PokerActs_from_game_.at[i,'New Card Events'] = 'Not Applicable'
                
            #Calling Events
            if hero_name_ in current_words and "calls" in current_words and PokerActs_from_game_.loc[i-1,'Call Stakes'] != '?':
                if float(PokerActs_from_game_.loc[i-1,'Call Stakes']) <= 2:
                    if float(PokerActs_from_game_.loc[i,'Win%']) < 0.35:
                        PokerActs_from_game_.at[i,'Calling Events'] = 'Loose Low-Risk Call'
                    if float(PokerActs_from_game_.loc[i,'Win%']) > 0.75:
                        PokerActs_from_game_.at[i,'Calling Events'] = 'Weak Low-Risk Call'
                elif float(PokerActs_from_game_.loc[i-1,'Call Stakes']) >= 10 and float(PokerActs_from_game_.loc[i-1,'Call Stakes']) <= 50:
                    if float(PokerActs_from_game_.loc[i,'Win%']) < 0.40:
                        PokerActs_from_game_.at[i,'Calling Events'] = 'Risky Call'
                    if float(PokerActs_from_game_.loc[i,'Win%']) > 0.75:
                        PokerActs_from_game_.at[i,'Calling Events'] = 'Strong Call'
                elif float(PokerActs_from_game_.loc[i-1,'Call Stakes']) > 50:
                    if float(PokerActs_from_game_.loc[i,'Win%']) < 0.50:
                        PokerActs_from_game_.at[i,'Calling Events'] = 'Bad Call'
                    if float(PokerActs_from_game_.loc[i,'Win%']) > 0.85:
                        PokerActs_from_game_.at[i,'Calling Events'] = 'Finishing Call'
                else:
                    PokerActs_from_game_.at[i,'Calling Events'] = 'Toss Up Call'
                    
            #Betting Events
            if hero_name_ in current_words and "bets" in current_words and PokerActs_from_game_.loc[i,'Relative Bet'] != '?':
                if float(PokerActs_from_game_.loc[i,'Win%']) < 0.35:
                    PokerActs_from_game_.at[i,'Bet/Bluff'] = 'Bluff'
                elif float(PokerActs_from_game_.loc[i,'Win%']) < 0.60:
                    PokerActs_from_game_.at[i,'Bet/Bluff'] = 'Semi-Bluff'
                else:
                    PokerActs_from_game_.at[i,'Bet/Bluff'] = 'Bet'
                    
            if hero_name_ in current_words and "bets" in current_words and PokerActs_from_game_.loc[i,'Relative Bet'] != '?':         
                if float(PokerActs_from_game_.loc[i,'Relative Bet']) <= 2:
                    if float(PokerActs_from_game_.loc[i,'Win%']) < 0.35:
                        PokerActs_from_game_.at[i,'Betting Events'] = 'Loose Low-Risk Bet'
                    if float(PokerActs_from_game_.loc[i,'Win%']) < 0.15:
                        PokerActs_from_game_.at[i,'Betting Events'] = 'Useless Bluff'
                    if float(PokerActs_from_game_.loc[i,'Win%']) > 0.75:
                        PokerActs_from_game_.at[i,'Betting Events'] = 'Weak Low-Risk Bet'
                if float(PokerActs_from_game_.loc[i,'Relative Bet']) >= 10 and float(PokerActs_from_game_.loc[i,'Relative Bet']) <= 50:
                    if float(PokerActs_from_game_.loc[i,'Win%']) < 0.50:
                        PokerActs_from_game_.at[i,'Betting Events'] = 'Risky Bet'
                    if float(PokerActs_from_game_.loc[i,'Win%']) < 0.35:
                        PokerActs_from_game_.at[i,'Betting Events'] = 'Risky Bluff'
                    if float(PokerActs_from_game_.loc[i,'Win%']) > 0.75:
                        PokerActs_from_game_.at[i,'Betting Events'] = 'Strong Bet'
                if float(PokerActs_from_game_.loc[i,'Relative Bet']) > 50:
                    if float(PokerActs_from_game_.loc[i,'Win%']) < 0.60:
                        PokerActs_from_game_.at[i,'Betting Events'] = 'Gutsy Bet'
                    if float(PokerActs_from_game_.loc[i,'Win%']) < 0.45:
                        PokerActs_from_game_.at[i,'Betting Events'] = 'Gutsy Bluff'
                    if float(PokerActs_from_game_.loc[i,'Win%']) > 0.75:
                        PokerActs_from_game_.at[i,'Betting Events'] = 'Finishing Bet'
                else:
                    PokerActs_from_game_.at[i,'Betting Events'] = 'Toss Up Bet'
                    
        #CRF Events
        current_line = PokerActs_from_game_.loc[i,'Log Line']
#         print("CURRENT LINE IS")
#         print(current_line)
        current_words = current_line.split()
        if villain_name_ in current_words and "bets" in current_words:
#             print("VILLAIN MADE A BET!")
#             print("At hand " + repr(PokerActs_from_game_.loc[i,'HandNum']))
#             print(current_words)
            next_line = PokerActs_from_game_.iloc[i+1,12]
            next_words = next_line.split()
#             print(next_words)
            if hero_name_ in next_words and "bets" in next_words:
                PokerActs_from_game_.at[i,'C/R/F'] = 'Raise Response'
            if hero_name_ in next_words and "folds." in next_words:
                PokerActs_from_game_.at[i,'C/R/F'] = 'Fold Response'
            if hero_name_ in next_words and "calls" in next_words:
                PokerActs_from_game_.at[i,'C/R/F'] = 'Call Response'
                
                    
        #Wins
        if hero_name_ in current_words and "wins" in current_words and "!" not in current_words:
            PokerActs_from_game_.at[i,'Win/Loss'] = 'Win'
        #Losses
        if villain_name_ in current_words and "wins" in current_words and "!" not in current_words:
            PokerActs_from_game_.at[i,'Win/Loss'] = 'Loss'
            
    return PokerActs_from_game_

In [10]:
subject_map = pd.read_csv('subject_map.csv')

# For each subject (this takes a long to run for all 30 – run 1 at a time to test)
for subject_num in range(1,30):
    print("STARTING SUBJECT NUMBER " + repr(subject_num))
    villain_name = subject_map.iloc[subject_num-1,3]
    hero_name = subject_map.iloc[subject_num-1,2]

    PokerActs_from_game = import_and_standardize_time_PokerActs(str(subject_num))
    FGs_from_game = import_and_standardize_time_FGs(str(subject_num))


    #Calculate the poker stats and identify the events in the game
    PokerActs_w_stats = calc_poker_stats(PokerActs_from_game, hero_name, villain_name)
    PokerActs_w_events = name_poker_events(PokerActs_w_stats, hero_name, villain_name)
    PokerActs_w_events

    poker_cols = PokerActs_w_events.columns.values
    FG_cols = FGs_from_game.columns.values

    #Merging the Poker Actions and Facial Gestures
    acts_and_FGs_cols = np.concatenate((FG_cols, poker_cols), axis=None)
    print(acts_and_FGs_cols)
    acts_and_FGs = pd.DataFrame(columns = acts_and_FGs_cols)
    acts_and_FGs
    df_index = 0
    
    #Merging FGs and Poker Acts using the timestamps
    for i in range(0, len(PokerActs_w_events)):
        start_time_index = PokerActs_w_events.loc[i, 'RelativeSec']
        if i < len(PokerActs_w_events)-1:
            j = i + 1
            end_time_index = PokerActs_w_events.loc[j, 'RelativeSec']
        else:
            j = len(PokerActs_w_events)-1
            end_time_index = PokerActs_w_events.loc[j, 'RelativeSec']
        
        temp_FG_df = FGs_from_game[(FGs_from_game['RelativeSec'] > start_time_index) & ((FGs_from_game['RelativeSec'] <= end_time_index) | (FGs_from_game['RelativeSec'] <= start_time_index))]
        
        if temp_FG_df.empty:
            d= {'01': [0], '02': [0], '04': [0], '05': [0], '06': [0], '07': [0], '09': [0], '10': [0], '12': [0], '14': [0], '15': [0], '17': [0], '20': [0], '23': [0], '25': [0], '26': [0], '45': [0], 'Inflection': [0], 'Onset Length': [0], 'Onset Unity': [0], 'Offset Length': [0], 'Offset Unity': [0], 'SumAmp': [0], 'TrueInflection': [0], 'RelativeTime': [0], 'RelativeSec': [0]}
            temp_FG_df = pd.DataFrame(data=d)

        for k in range(0, len(temp_FG_df)):
            df_index += 1

            new_row = np.concatenate((temp_FG_df.iloc[k,:].values, PokerActs_w_events.iloc[i,:].values), axis = None)

            acts_and_FGs.loc[df_index] = new_row
    acts_and_FGs
    acts_and_FGs.to_csv(str(subject_num)+ "_acts_and_FGs_NEW(PFLP,CRF).csv")

STARTING SUBJECT NUMBER 1
GETTING NEW WIN % at phase 'FLOP'


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


0.3989328997654622
GETTING NEW WIN % at phase 'TURN'
0.35990338164251207
GETTING NEW WIN % at phase 'RIVER'
0.45454545454545453
GETTING NEW WIN % at phase 'FLOP'
0.4229604088993543
GETTING NEW WIN % at phase 'TURN'
0.33535353535353535
GETTING NEW WIN % at phase 'RIVER'
0.2585858585858586
GETTING NEW WIN % at phase 'FLOP'
0.4223464992197647
GETTING NEW WIN % at phase 'TURN'
0.548792270531401
GETTING NEW WIN % at phase 'RIVER'
0.6666666666666666
GETTING NEW WIN % at phase 'FLOP'
0.29829469533447334
GETTING NEW WIN % at phase 'TURN'
0.17404479578392623
GETTING NEW WIN % at phase 'RIVER'
0.04040404040404041
GETTING NEW WIN % at phase 'FLOP'
0.5699744905110308
GETTING NEW WIN % at phase 'TURN'
0.5323451910408432
GETTING NEW WIN % at phase 'RIVER'
0.593939393939394
GETTING NEW WIN % at phase 'FLOP'
0.6595043870714546
GETTING NEW WIN % at phase 'TURN'
0.6170838823012736
GETTING NEW WIN % at phase 'RIVER'
0.5686868686868687
GETTING NEW WIN % at phase 'FLOP'
0.3750418150048122
GETTING NEW WIN %

KeyboardInterrupt: 